In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import ProtoKartering

### Access kartering inladen

De aangeleverde SBB typen worden opgeschoond en vervolgens door het aanmaken van SBB instances voor in de VegTypeInfo gevalideerd. De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [2]:
shape_path = Path("../testing/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../testing/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013/")
shape_elm_id_column = "ElmID"

access_kartering = ProtoKartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.iloc[:3]


,ElmID,Datum,Projcode,Opp,Dlgebied,Opmerking,VegType,LabelKaart,geometry,intern_id,Locatie,VegTypeInfo
0,28049,19072013,864,0.376174,RM-Kern,None,"09A2-6.e,h,v.90",09A2-6(2),"POLYGON ((189433.818 538314.302, 189439.215 53...",1809,1809,"[(90%, VvN: [], SBB: ['9a2a']), (10%, VvN: [],..."
1,15034,10092013,864,1.044266,RM-Oost,None,50A-1.e,50A-1,"POLYGON ((190568.500 539171.122, 190538.023 53...",815,815,"[(100%, VvN: [], SBB: ['50a'])]"
2,13503,14102013,864,0.217410,RM-Noord,None,"09A2-1.e,h,q,v.90",09A2-1(2),"POLYGON ((189538.104 539409.332, 189532.399 53...",650,650,"[(90%, VvN: [], SBB: ['9a2a']), (10%, VvN: [],..."


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [3]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)

Alle SBB codes zijn valide
Alle VvN codes zijn valide


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


In [4]:
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(20)

Alle SBB codes zijn valide
Alle VvN codes zijn valide


,VvN,SBB
0,14rg10,14-l
1,None,7-a
2,None,8b-b
3,None,11/c
4,None,11a-b
5,None,14/f
6,None,19-f
7,None,19-h
8,None,20a1c
9,None,33a7


### Toevoegen VvN met Was-Wordt lijst aan Access kartering 

Hier wordt aan elke VegTypeInfo de VvN toegevoegd die passen bij de SBB die al aanwezig was.

In [5]:
vegtypeinfo_series = access_kartering.gdf.VegTypeInfo.apply(wwl.toevoegen_VvN_aan_List_VegTypeInfo)
print(vegtypeinfo_series)

0       [(90%, VvN: ['9aa2a'], SBB: ['9a2a']), (10%, V...
1                         [(100%, VvN: [], SBB: ['50a'])]
2       [(90%, VvN: ['9aa2a'], SBB: ['9a2a']), (10%, V...
3       [(80%, VvN: [], SBB: ['9b-a']), (10%, VvN: ['9...
4       [(58%, VvN: ['8bb4d'], SBB: ['8b3c']), (29%, V...
                              ...                        
1944    [(90%, VvN: ['36aa2a'], SBB: ['36a2']), (10%, ...
1945    [(90%, VvN: ['32rg3'], SBB: ['32-c']), (10%, V...
1946    [(60%, VvN: ['8ba2'], SBB: ['8c1c']), (30%, Vv...
1947    [(90%, VvN: ['5bb1'], SBB: ['5c1']), (10%, VvN...
1948             [(100%, VvN: ['36aa2a'], SBB: ['36a2'])]
Name: VegTypeInfo, Length: 1949, dtype: object


In [6]:
vegtypeinfo_series.iloc[150]

[VegTypeInfo(percentage=100, SBB=[SBB(klasse='50', verbond='a', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap=None, max_match_level=2)], VvN=[])]

In [7]:
# Example of 3 vegtypeinfos in 1 geometry
vegtypeinfo_series.iloc[3]

[VegTypeInfo(percentage=80, SBB=[SBB(klasse='9', verbond='b', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a', max_match_level=1)], VvN=[]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='g', max_match_level=1)], VvN=[VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3', max_match_level=1)]),
 VegTypeInfo(percentage=10, SBB=[SBB(klasse='9', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None, max_match_level=4)], VvN=[VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None, max_match_level=5)])]

### Definitietabel opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [8]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_out_dt)

Alle SBB codes zijn valide
Alle VvN codes zijn valide


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


In [9]:
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head(20)

,Habitattype,Kwaliteit,SBB,VvN,mits,mozaiek
0,H1130,GoedMatig.GOED,None,2aa1,mits in de FGR Getijdengebied en mits gelegen ...,<NA>
1,H1130,GoedMatig.GOED,None,2aa2,mits in de FGR Getijdengebied en mits gelegen ...,<NA>
2,H1130,GoedMatig.GOED,None,3aa1,mits in de FGR Getijdengebied en mits gelegen ...,<NA>
3,H1130,GoedMatig.GOED,None,3aa2,mits in de FGR Getijdengebied en mits gelegen ...,<NA>
4,H1140_A,GoedMatig.GOED,None,2aa1,mits in de slikkige en fijnzandige delen van F...,<NA>
5,H1140_A,GoedMatig.GOED,None,3aa1,mits in de slikkige en fijnzandige delen van F...,<NA>
6,H1140_A,GoedMatig.GOED,None,3aa2,mits in de slikkige en fijnzandige delen van F...,<NA>
7,H1160,GoedMatig.GOED,None,2aa1,mits in de FGR Getijdengebied en mits in een i...,<NA>
8,H1160,GoedMatig.GOED,None,3aa1,mits in de FGR Getijdengebied en mits in een i...,<NA>
9,H1160,GoedMatig.GOED,None,3aa2,mits in de FGR Getijdengebied en mits in een i...,<NA>


### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [10]:
habitat_voorstellen = vegtypeinfo_series.apply(lambda infos: [dt.find_habtypes(info) for info in infos])
habitat_voorstellen

0       [[HabitatVoorstel(vegtype=VvN(klasse='9', orde...
1                                                    [[]]
2       [[HabitatVoorstel(vegtype=VvN(klasse='9', orde...
3       [[HabitatVoorstel(vegtype=SBB(klasse='9', verb...
4       [[], [HabitatVoorstel(vegtype=VvN(klasse='8', ...
                              ...                        
1944    [[HabitatVoorstel(vegtype=VvN(klasse='36', ord...
1945                                             [[], []]
1946    [[HabitatVoorstel(vegtype=VvN(klasse='8', orde...
1947    [[HabitatVoorstel(vegtype=VvN(klasse='5', orde...
1948    [[HabitatVoorstel(vegtype=VvN(klasse='36', ord...
Name: VegTypeInfo, Length: 1949, dtype: object

In [11]:
habitat_voorstellen[150]

[[]]

In [12]:
habitat_voorstellen[3]

[[HabitatVoorstel(vegtype=SBB(klasse='9', verbond='b', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a', max_match_level=1), habtype='H3160', kwaliteit=<GoedMatig.GOED: 'Goed'>, regel_in_deftabel=306, mits=None, mozaiek=None, match_level=1)],
 [HabitatVoorstel(vegtype=VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3', max_match_level=1), habtype='H2190_C', kwaliteit=<GoedMatig.MATIG: 'Matig'>, regel_in_deftabel=200, mits=None, mozaiek=None, match_level=1)],
 [HabitatVoorstel(vegtype=VvN(klasse='9', orde='a', verbond='a', associatie='2', subassociatie='b', derivaatgemeenschap=None, rompgemeenschap=None, max_match_level=5), habtype='H7140_B', kwaliteit=<GoedMatig.GOED: 'Goed'>, regel_in_deftabel=520, mits=None, mozaiek=None, match_level=4)]]